In [5]:
using JuMP
using Cbc
using DataFrames
using Gurobi
using CSV

In [15]:
# m = Model(solver = CbcSolver())
m = Model(solver=GurobiSolver(Presolve=0))

data = CSV.read("supermarket_data.csv")
simple_dist = CSV.read("distmatrix_simple.csv")

v = data[:,2]

simple_dist = simple_dist[:,2:103]
c = simple_dist
    
@variables m begin
    y[1:102], Bin
end

@variables m begin
    x[1:102,1:102], Bin
end

@variables m begin
    z[1:102]
end

@variables m begin
    t[1:102,1:102]
end

@constraint(m, y[1] == 1)
@constraint(m, y[102] == 1)
@constraint(m, z[1] == 0)
@constraint(m, sum(y[i] for i in 2:101) <= 15)

for i in 1:101
    @constraint(m, sum(x[i,j] for j in 1:102) == y[i])
end

for j in 2:102
    @constraint(m, sum(x[i,j] for i in 1:102) == y[j])
end

for j in 2:102
    @constraint(m, sum(t[i,j] for i in 1:102) == z[j])
end

for j in 1:101
    @constraint(m, sum(t[j,k] for k in 1:102) == (z[j] + sum(c[j,k]x[j,k] for k in 1:102)))
end

for j in 1:102
    for k in 1:102
        @constraint(m, t[j,k] >= 0)
        @constraint(m, t[j,k] <= 90x[j,k])
    end
end

@objective(m, Max, sum(v[i]*y[i] for i in 1:102))


solve(m)

for i in 2:102
    println(getvalue(y[i]))
end
println("ANSWER")
println("Total Distance: ", getobjectivevalue(m))


Academic license - for non-commercial use only
Optimize a model with 21216 rows, 21012 columns and 83228 nonzeros
Variable types: 10506 continuous, 10506 integer (10506 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+01]
  Objective range  [9e-01, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective -0.0000000
Variable types: 10505 continuous, 10507 integer (10506 binary)

Root relaxation: objective 1.629900e+02, 11252 iterations, 0.11 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  162.99000    0    3   -0.00000  162.99000      -     -    0s
     0     0  162.99000    0    3   -0.00000  162.99000      -     -    0s
H    0     0                     150.0000000  162.99000  8.66%     -    0s
H    0     0                     162.9900000  162.99000  0.00%     -    0s

Explored 1 nodes (16245 sim